In [4]:
#!/usr/bin/python3

from motion_control import Planner
import cv2
import visual_servo
import config
from mango_detection import mango_detector as mod
import tensorflow as tf

In [5]:
camera_template = {
    'device' : cv2.VideoCapture(config.CAMERA_END_EFFECTOR),
    'focus_length' : config.camera_focus_length,
    'offset' : [0, 0, 0], # x, y, z
}

tree_position = [1000, 3000, 5000]
base_position = [
    (500, 90),
    (1500, 270),
    (2500, 90),
    (3500, 270),
    (4500, 90),
    (5500, 270),
]
drop_position = {
    "Green": [0],
    "Yellow": [180],
}

planner = Planner.getInstance()
current_pos = [0, 0, 0]

mango_model = mango_detector.load_model('mango_detection/model/')

In [10]:
def get_distance(pos1, pos2):
    def diff(a1, a2):
        return np.abs(a1 - a2)
    return np.sqrt(diff(pos1[0], pos2[0])**2 + diff(pos1[1], pos2[1])**2 + diff(pos1[2], pos2[2])**2)
        
def state_wait_move(target):
    current_pos = planner.get_pos()
    return True if get_distance(current_pos, target) <= config.accept_move else False

def state_scan(image_height, image_width, camera, i, j):
    workspace_x, workspace_y = get_workspace(image_width, image_heigth, camera)
    x_pos = workspace_x / 2 + workspace_x * j
    y_pos = workspace_y / 2 + workspace_y * i
    
    target = [x_pos, y_pos, current_pos[2]]
    planner.move_to_stero_cam(target[0], target[1], target[2], config.default_speed)
    return state_wait_move(target)

def state_find_deg_maxscore(tf_session, mango_model, camera):
    frame = camera['device'].read()
    if not frame[0]:
        print ("Camera Failed")
        return False
        
    frame = frame[1]
    image_height, image_width, ch = frame.shape
    result = md.detect(frame, mango_model[1:], sess, 0.7)
    if 'center' in result.keys():
        depth = planner.get_depth()
        x_len = pixel_to_distance(camera, (image_width / 2 - result['center'][0]), depth)
        deg_go = x_len / planner.get_arm_length()        
        planner.turn_arm(deg_go, config.default_speed)
        
        return True if x_len < config.accept_move else False
    return False

def turn_arm_drop_mango(color = 1):
    if color not in [1, 2]:
        color = 1
    planner.turn_to_arm(config.basket[color])
    
    return True if np.abs(planner.get_arm_deg() - config.basket[color]) <= 2 else False

def drop_mango():
    planner.drop_mango(True)
    time.sleep(1.5)
    planner.drop_mango(False)
    
    return True

def cut_mango():
    planner.cut_mango(True)
    time.sleep(1.5)
    planner.cut_mango(False)
    
    return True

In [3]:
if __name__ == '__main__':
    mango_model = md.load_model('mango_detection/model/', 'mango_2.0.pb')
    detection_graph, tensor_dict, image_tensor, category_index = mango_model
    camera = camera_template
    
    image_height, image_width = [720, 1280]    
    (i_workspace, j_workspace) = get_workspace(image_width, image_heigth, camera)
    count_j = config.workspace_x / j_workspace # x
    count_i = config.workspace_y / i_workspace # y
    
    with detection_graph.as_default():
        with tf.Session() as sess:
            camera['device'].start()
            for index, item in enumberate(base_position):
                for i in range(count_i):
                    for j in range(count_j):
                        state = 0
                        while True:
                            if state == 0:
                                if state_scan(image_height, image_width, camera, i, j):
                                    state = 1:
                            elif state == 1:
                                frame = camera['device'].read()
                                    if not frame[0]:
                                        print ("Camera Fail")
                                        continue
                                    result = md.detect(frame[1], mango_model[1:], sess, 0.7)
                                    if 'center' in result.keys():
                                        state = 2
                                    else:
                                        print ("Not Found Mango")
                                        break
                            elif state == 2:
                                if state_find_deg_maxscore(sess, magno_model[1:], camera)
                                    vs = visual_servo.VisualServo(True)
                                    vs.set_detector(sess, mango_model)
                                    vs.set_camera(camera['device'])
                                    try:
                                        vs.start()
                                    finally:
                                        vs.stop()
                                    
                                    state = 3
                            
                            elif state == 3:
                                cut_mango()
                                state = 4
                            
                            elif state == 4:
                                if turn_arm_drop_mango(1):
                                    state = 5
                            
                            elif state == 5:
                                drop_mango()
                                state = 6
                                
                            elif state == 6:
                                planner.turn_to_arm(item[1])
                                if np.abs(planner.get_arm_deg() - item[1]) <= 2:
                                    state = 1
                            
                            print (state)
                
    camera['device'].stop()